In [ ]:
#%pip install duckdb

In [22]:

%run generate_report.py --season-year 2026 --no-logbook-fetch

Skipping data download (--no-logbook-fetch specified)
Setting up database connection...
Generating PDF report...
Creating pace over time chart...
Saving pace chart to PDF...
Creating monthly meters chart...
Generating PDF report...
Creating pace over time chart...
Saving pace chart to PDF...
Creating monthly meters chart...
PDF report generated successfully: concept2_workout_report.pdf
PDF report generated successfully: concept2_workout_report.pdf


In [2]:
import duckdb
cn = duckdb.connect()
cn.sql("create or replace view v_detail_workouts as select * from read_csv_auto('~/concept2/workouts/*detail*.csv')")
cn.sql("create or replace view v_summary_workouts as select * from read_csv_auto('~/concept2/workouts/*summary*.csv')")    

In [8]:
sql = """

    select 
        "date" as workout_date
        , "Time (seconds)" as time_seconds
        , "Distance (meters)" as distance_meters
        ,format('{:02d}:{:02d}', 
            cast(floor("Time (seconds)" / 60) as int), 
            cast(round("Time (seconds)") % 60 as int)
        ) as time_mmss
    from v_detail_workouts
    where 1=1
        and machine_type = 'rowerg'
        and "Distance (meters)" >= 5000
    qualify 1 = rank() over(partition by "date" order by "Number")
    order by "Time (seconds)" 
    limit 20

"""
cn.sql(sql).show()

┌──────────────┬──────────────┬─────────────────┬───────────┐
│ workout_date │ time_seconds │ distance_meters │ time_mmss │
│     date     │    double    │     double      │  varchar  │
├──────────────┼──────────────┼─────────────────┼───────────┤
│ 2025-03-13   │       1126.2 │          5005.3 │ 18:46     │
│ 2025-06-19   │       1145.0 │          5002.6 │ 19:05     │
│ 2025-03-04   │       1147.9 │          5000.0 │ 19:08     │
│ 2025-05-14   │       1149.0 │          5008.8 │ 19:09     │
│ 2025-04-04   │       1150.8 │          5003.9 │ 19:11     │
│ 2024-11-04   │       1151.7 │          5006.1 │ 19:12     │
│ 2025-01-25   │       1156.4 │          5008.6 │ 19:16     │
│ 2025-01-07   │       1164.2 │          5005.5 │ 19:24     │
│ 2025-01-23   │       1165.0 │          5002.6 │ 19:25     │
│ 2025-02-20   │       1165.0 │          5007.0 │ 19:25     │
│ 2025-03-17   │       1165.8 │          5005.8 │ 19:26     │
│ 2024-12-04   │       1167.7 │          5002.5 │ 19:28     │
│ 2024-1